In [1]:
%reset
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

In [2]:
MAX_FEATURES = 10000
MAX_LEN = 20

In [5]:
%%time
# Get data.
(x_train, y_train), (x_test, y_test) = (
    tf.keras.datasets.imdb.load_data(num_words=MAX_FEATURES))

CPU times: user 4.59 s, sys: 265 ms, total: 4.85 s
Wall time: 4.86 s


In [8]:
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=MAX_LEN)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=MAX_LEN)

In [11]:
def get_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Embedding(10000, 8, input_length=MAX_LEN))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    model.compile(
        optimizer='rmsprop',
        loss='binary_crossentropy',
        metrics=['acc'])
    model.summary()
    return model
    
model = get_model()
history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 8)             80000     
_________________________________________________________________
flatten_3 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________
Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 1s 57us/step - loss: 0.7026 - acc: 0.5208 - val_loss: 0.6893 - val_acc: 0.5406
Epoch 2/10
20000/20000 [==============================] - 1s 44us/step - loss: 0.6663 - acc: 0.5922 - val_loss: 0.6561 - val_acc: 0.6110
Epoch 3/10
20000/20000 [==============================] - 1s 43us/step - loss: 

array([1, 0, 0, ..., 0, 1, 0])